# E4 Lyrics
* 소설을 학습시켜 글을 스스로 쓰는 모델 생성
* LSTM과 Embedding사용하여 모델 작성
* 문자열 처리와 토큰 사용 방법 연습

## 00 package import and fuction define

In [1]:
import os, re
import numpy as np
import glob
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [2]:
def preprocess_sentence(sentence):
    import re
    sentence = sentence.lower().strip() #1
    ssentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


#### preprocess sentence
1. 문자 한문장을 입력받아
2. 한문장 내의 특수문자를 해결
    1. str.lower(): 해당 문자열을 소문자로 변경
    2. str.strip([chars]): 인자로 전달된 문자를 str의 양쪽에서 제거
    3. re.sub(a,b,sentence) :  sentence의 a를 b로 수정<br/>
        r은 패턴을 명시할 때 r을 사용하고 뒤에 패턴을 표시<br/>
        re.sub(r"([?.!,¿])", r" \1 ", sentence) : 특수문자 양쪽에 공백 추가<br/>
        re.sub(r'[" "]+', " ", sentence) : 여러개의 공백은 하나의 공백으로<br/>
        re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) : a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로<br/>
    4. 다시 양쪽 공백을 지웁니다
3. 각 문장의 시작과 끝을 표시
    <start>, <end>를 문장 양 끝에 표시
4. 처리된 문장을 리턴
    
**정규표현식**
    r은 뒤에 나오는 문자열을 패턴으로 인식
    \1은 하나의 문자
    +는 하나이상
    ^은 아님의 의미 
    r[" "]은 " "의 패턴으로 이루어진 모든 문자
    r"[^a-zA-Z?.!,¿]+" : 소문자 a부터 z까지 대문자 A부터Z까지 문장기호가 아닌 것들이 하나 이상 
    

In [3]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus로 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게함
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  
    print(tensor,tokenizer) 
    return tensor, tokenizer

#### tokenize
1. tensorflow의 Tokenizer를 이용해 문자열의 사전생성
    tokenizer(num_words, filters, oov_token) : filter로 분할된 문자열을 최대 num_words만큼 입력받아 각 문자열에 번호를 붙이는 토큰화 함수 객체 생성, 사전에 없는 단어는 oov_token으로 해결
2. 문자열을 입려해 사전 완성
3. 완성된 사전으로 문자열을 토큰화 한 뒤 tensor로 변환
4. 입력 데이터의 길이를 일정하게 조정

In [4]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

#### textgenerator
* 텐서플로 패키지를 이용해 모델구축
* embedding : 문자열 엠베딩 int로 이루어진 array형태를 입력받는 레이어
* lstm : long short term memory rnn계열의 순환신경망으로 여기서는 문자열을 출력하기 위해 각 순환 단계마다의 cellstate를 산출 
* dense : 각 단어 엠베딩에 할당하기위해 사용 기본적인 텐서플로우의 레이어

In [5]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]


    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

#### generate_text
* 학습된 모델과 toenizer(토큰화 시키는 함수 객체)를 입력받아 텍스트를 생성하는 함수
* init_sentence는 시작하는 문자열 max_len은 생성되는 문자열의 최대 길이
    1. init_sentence를 입력받아 토큰화 함수 객체를 이용해 토큰화 및 텐서화
    while
    2. 입력받은 텐서를 모델에 입력 
    3. 모델에서 예측된 값 중 가장 높은 확률인 word index를 산출
    4. 3에서 예측된 word index를 문장 뒤에 붙임
    5. 모델이 <end>를 예측했거나, max_len에 도달했다면 while break
* 텐서화 된 토큰들을 다시 문자열로 변경후 출력

## data_set load and preprocessing

In [6]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


텍스트 파일 불러들여서 리스트에 저장하기
1. 경로의 밑의 파일을 glob을 이용해서 리스트에 저장
2. 리스트에 저장된 파일들을 for loop를 이용해서 각 파일을 한줄씩 읽어 raw_corpus에 한줄씩 저장 

In [7]:
print(preprocess_sentence("he's a ;;;sample        sentence."))

<start> he s a sample sentence. <end>


In [8]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0 : continue
    if sentence[-1] == ":": continue
    if len(sentence) > 15 : continue
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
corpus[:10]

['<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> she tied you <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> in every word <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> and even though <end>',
 '<start> hallelujah <end>']

문자열이 저장에 리스트 전처리하기

특정 조건의 문자열을 제외하고 특수문자 문장부호 및 소문자 대문자 처리<br/>
    1. 길이가 0인 경우 <br/>
    2. 문자열이 마지막에 :로 끝나는경우<br/>
    3. 문자열의 길이가 15 이상인 경우<br/>

In [9]:
tensor, tokenizer = tokenize(corpus)

[[   2  155    3 ...    0    0    0]
 [   2  155    3 ...    0    0    0]
 [   2   49  917 ...    0    0    0]
 ...
 [   2  210 2041 ...    0    0    0]
 [   2   55   14 ...    0    0    0]
 [   2   30  387 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe095d64700>


tokenize를 이용해서 토큰화 함수객체와 토큰화 된 문자열 데이터를 생성

In [10]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : you
6 : it
7 : me
8 : oh
9 : a
10 : the


In [11]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]  

NLP에서 input 문장을 Source sentence, output 문장을 Target sentence로 지칭<br/>
여기서는 start부터 end 이전까지를 src, start 다음 문자부터 end까지를 tgt로 세팅

In [12]:
x_train,x_test,y_train,y_test = train_test_split(src_input,tgt_input, test_size = 0.2)

학습을 위한 train test 분리

In [13]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
test_dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

여기서는 tensorflow에서 제공하는 Dataset 형식을 사용
일반적으로 잘 학습하기위해 셔플 실행
bacth 사이즈로 해당 데이터를 구성하되 마지막 데이터셋이 batch보다 작으면 drop 

In [14]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


* tokenizer가 구축한 단어사전 내 포함되지 않은 0:<pad>를 포함하여 +1개

In [16]:
for src_sample, tgt_sample in train_dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.34230286e-04,  1.72765271e-04,  9.35374555e-05, ...,
         -2.16025845e-04,  8.58165658e-05,  2.73018173e-04],
        [ 3.73696035e-04,  1.79407783e-04,  2.21729497e-04, ...,
         -2.83687201e-04,  4.40890981e-05,  4.01186175e-04],
        [ 5.96163212e-04,  9.78819298e-05,  3.40118801e-04, ...,
         -2.91243661e-04, -5.20155299e-05,  4.79569630e-04],
        ...,
        [ 1.31632015e-03,  2.73205747e-04,  2.85118213e-03, ...,
          1.76362728e-03,  8.39839398e-04,  1.10501796e-03],
        [ 1.45802449e-03,  2.84484180e-04,  3.03320982e-03, ...,
          1.86243374e-03,  9.75095492e-04,  1.14837626e-03],
        [ 1.59663160e-03,  3.01058870e-04,  3.18692368e-03, ...,
          1.93339481e-03,  1.09620905e-03,  1.18778297e-03]],

       [[ 1.34230286e-04,  1.72765271e-04,  9.35374555e-05, ...,
         -2.16025845e-04,  8.58165658e-05,  2.73018173e-04],
        [ 3.29840375e-04,  1.02772334e-04,  3

In [19]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [18]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, epochs=10)

Epoch 1/10
44/44 [==============================] - 9s 149ms/step - loss: 2.6831
Epoch 2/10
44/44 [==============================] - 7s 152ms/step - loss: 1.4976
Epoch 3/10
44/44 [==============================] - 7s 154ms/step - loss: 1.2589
Epoch 4/10
44/44 [==============================] - 7s 157ms/step - loss: 1.1793
Epoch 5/10
44/44 [==============================] - 7s 159ms/step - loss: 1.1455
Epoch 6/10
44/44 [==============================] - 7s 160ms/step - loss: 1.1249
Epoch 7/10
44/44 [==============================] - 7s 163ms/step - loss: 1.1083
Epoch 8/10
44/44 [==============================] - 7s 163ms/step - loss: 1.0893
Epoch 9/10
44/44 [==============================] - 7s 165ms/step - loss: 1.0707
Epoch 10/10
44/44 [==============================] - 7s 169ms/step - loss: 1.0532


In [26]:
generate_text(model, tokenizer, init_sentence="<start> where")

'<start> where i t <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

In [38]:
generate_text(model, tokenizer, init_sentence="<start> i have", max_len=20)

'<start> i have a da <end> '

# 회고
* 학습시키는 텍스트에 질과 내용에 따라 모델의 정확도 및 어투가 달라지는 것을 확인
* embedding 및 모데을 구조에 따른 loss 및 예측 결과 확인 필요
* 모델이 굉장히 heavy해서 최적화 시킬수 있는 방법 확인 필요
* validation 및 여러 regulation을 적용 해볼 필요
* 순방향 뿐만 아니라 양방향 신경망에서의 적용 필요